## Table 3 and 4 (Journal of Climate submission; Molina et al.)

Table 3. ONI variance (◦C2) for different time periods of the Global and Pacific freshwater hosing experimentsas compared to the CESM1 control. Years 201-500 include the time period of AMOC collapse for the sensitivityexperiments. Years 551-650 contain the time period when the 0.2 Sv experiments overshoot AMOC strength inthe CESM1 control and 0.4 Sv experiments contain a mostly collapsed AMOC state. Years 651-800 contain theAMOC overshoot period for the 0.4 Sv experiment.

Table 4. ONI variance (◦C2) for different time periods of the Pacific Salt experiment as compared to theCESM1 control. Years 101-250 include the time period of PMOC intensification and years 251-300 contain thetime period of rapid PMOC weakening and return to a PMOC collapsed state in the Pacific Salt experiment.

**Figure by: Maria J. Molina, NCAR**

In [1]:
# imports 
import xarray as xr
import pandas as pd
import numpy as np
from climatico import enso
from climatico.util import weighted_mean
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from datetime import timedelta

In [2]:
from config import directory_figs, directory_data

In [3]:
# list of filenames to do this for
file_g02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.*.nc'
file_g04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG04Sv.pop.h.SST.*.nc'
file_p02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP02Sv.pop.h.SST.*.nc'
file_p04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP04Sv.pop.h.SST.*.nc'
file_psalt = 'b1d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.*.nc'
file_cntrl = 'b1d.e11.B1850C5CN.f09_g16.005.pop.h.SST.*.nc'

In [4]:
nino = enso.DefineNino(nino='nino34', lats='lat', lons='lon', cutoff=0.5, runningmean=3)

In [5]:
# extract first 100 years from control run for concatenating
ds = xr.open_mfdataset(f'{directory_data}{file_cntrl}',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds = ds.assign_coords(time=ds.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds = ds.isel(z_t=0)
# compute weighted mean of sst region
first100_ssts = ds['SST']

In [6]:
# control run index computation
ds2 = xr.open_mfdataset(f'{directory_data}{file_cntrl}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds2 = ds2.assign_coords(time=ds2.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds2 = ds2.isel(z_t=0)

# attach first 100 years
control_ssts = xr.concat([first100_ssts.sel(time=slice('0700-01-01 00:00:00', '0800-01-01 00:00:00')),
                          ds2['SST'].sel(time=slice('0800-01-01 00:00:00', '1600-01-01 00:00:00'))], dim='time')
control_ssts['time'] = xr.cftime_range(start="0001-01-01 00:00:00", end="0900-12-31 00:00:00", freq="MS", calendar="noleap")

# rolling climo 
control_ssts_roll = nino.monthly_climo(control_ssts.chunk({'time':900}), yrsroll=30, centered=True, time='time')

# compute nino index
control_nino, stdval = nino.compute_index(
                                  control_ssts, control_ssts_roll, 
                                  linear_detrend=False, lat_name='lat',
                                  output_std=True, input_std=None, custom_std=False, first_threshold=100, last_threshold=900)                               

# grab numpy array
#control_nino = control_nino.values
control_stdval = stdval.values

In [7]:
# g02sv index computation
ds3 = xr.open_mfdataset(f'{directory_data}{file_g02sv}',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds3 = ds3.assign_coords(time=ds3.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds3 = ds3.isel(z_t=0)

# attach first 100 years
g02sv_ssts = xr.concat([first100_ssts.sel(time=slice('0700-01-01 00:00:00', '0800-01-01 00:00:00')),
                        ds3['SST'].sel(time=slice('0001-01-01 00:00:00', '0801-01-01 00:00:00'))], dim='time')
g02sv_ssts['time']=xr.cftime_range(start="0001-01-01 00:00:00", end="0900-12-31 00:00:00", freq="MS", calendar="noleap")

# rolling climo 
g02sv_ssts_roll = nino.monthly_climo(g02sv_ssts.chunk({'time':900}), yrsroll=30, centered=True, time='time')

# compute nino index
g02sv_nino = nino.compute_index(g02sv_ssts, g02sv_ssts_roll, 
                                  linear_detrend=False, lat_name='lat',
                                  output_std=False, input_std=control_stdval, custom_std=True, first_threshold=100, last_threshold=900)

# grab numpy array
#g02sv_nino = g02sv_nino.values

In [8]:
# g04sv index computation
ds4 = xr.open_mfdataset(f'{directory_data}{file_g04sv}',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds4 = ds4.assign_coords(time=ds4.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds4 = ds4.isel(z_t=0)

# attach first 100 years
g04sv_ssts = xr.concat([first100_ssts.sel(time=slice('0700-01-01 00:00:00', '0800-01-01 00:00:00')),
                        ds4['SST'].sel(time=slice('0001-01-01 00:00:00', '0801-01-01 00:00:00'))], dim='time')
g04sv_ssts['time']=xr.cftime_range(start="0001-01-01 00:00:00", end="0900-12-31 00:00:00", freq="MS", calendar="noleap")

# rolling climo 
g04sv_ssts_roll = nino.monthly_climo(g04sv_ssts.chunk({'time':900}), yrsroll=30, centered=True, time='time')

# compute nino index
g04sv_nino = nino.compute_index(g04sv_ssts, g04sv_ssts_roll, 
                                  linear_detrend=False, lat_name='lat',
                                  output_std=False, input_std=control_stdval, custom_std=True, first_threshold=100, last_threshold=900)

# grab numpy arrays
#g04sv_nino = g04sv_nino.values

In [9]:
# p02sv index computation
ds5 = xr.open_mfdataset(f'{directory_data}{file_p02sv}',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds5 = ds5.assign_coords(time=ds5.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds5 = ds5.isel(z_t=0)

# attach first 100 years
p02sv_ssts = xr.concat([first100_ssts.sel(time=slice('0700-01-01 00:00:00', '0800-01-01 00:00:00')),
                        ds5['SST'].sel(time=slice('0001-01-01 00:00:00', '0801-01-01 00:00:00'))], dim='time')
p02sv_ssts['time'] = xr.cftime_range(start="0001-01-01 00:00:00", end="0900-12-31 00:00:00", freq="MS", calendar="noleap")

# rolling climo 
p02sv_ssts_roll = nino.monthly_climo(p02sv_ssts.chunk({'time':900}), yrsroll=30, centered=True, time='time')

# compute nino index
p02sv_nino = nino.compute_index(p02sv_ssts, p02sv_ssts_roll, 
                                  linear_detrend=False, lat_name='lat',
                                  output_std=False, input_std=control_stdval, custom_std=True, first_threshold=100, last_threshold=900)

# grab numpy arrays
#p02sv_nino = p02sv_nino.values

In [10]:
# p04sv index computation
ds6 = xr.open_mfdataset(f'{directory_data}{file_p04sv}',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds6 = ds6.assign_coords(time=ds6.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds6 = ds6.isel(z_t=0)

# attach first 100 years
p04sv_ssts = xr.concat([first100_ssts.sel(time=slice('0700-01-01 00:00:00', '0800-01-01 00:00:00')),
                        ds6['SST'].sel(time=slice('0001-01-01 00:00:00', '0801-01-01 00:00:00'))], dim='time')
p04sv_ssts['time']=xr.cftime_range(start="0001-01-01 00:00:00", end="0900-12-31 00:00:00", freq="MS", calendar="noleap")

# rolling climo 
p04sv_ssts_roll = nino.monthly_climo(p04sv_ssts.chunk({'time':900}), yrsroll=30, centered=True, time='time')

# compute nino index
p04sv_nino = nino.compute_index(p04sv_ssts, p04sv_ssts_roll, 
                                  linear_detrend=False, lat_name='lat',
                                  output_std=False, input_std=control_stdval, custom_std=True, first_threshold=100, last_threshold=900)

# grab numpy arrays
#p04sv_nino = p04sv_nino.values

In [11]:
# psalt index computation
ds7 = xr.open_mfdataset(f'{directory_data}{file_psalt}',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds7 = ds7.assign_coords(time=ds7.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds7 = ds7.isel(z_t=0)

# psalt index computation
ds8 = xr.open_mfdataset(f'{directory_data}b2d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.030101-035012.nc',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds8 = ds8.assign_coords(time=ds8.coords['time'] - timedelta(days=17))
ds8 = ds8.sel(time=slice('0301-01-01 00:00:00', '0351-01-01 00:00:00'))

# attach first 100 years
psalt_ssts = xr.concat([first100_ssts.sel(time=slice('0700-01-01 00:00:00', '0800-01-01 00:00:00')).drop('z_t'),
                        ds7['SST'].sel(time=slice('0001-01-01 00:00:00', '0301-01-01 00:00:00')).drop('z_t'),
                        ds8['SST']], dim='time')
psalt_ssts['time']=xr.cftime_range(start="0001-01-01 00:00:00", end="0450-12-31 00:00:00", freq="MS", calendar="noleap")

# rolling climo 
psalt_ssts_roll = nino.monthly_climo(psalt_ssts.chunk({'time':900}), yrsroll=30, centered=True, time='time')

# compute nino index
psalt_nino = nino.compute_index(psalt_ssts, psalt_ssts_roll, 
                                  linear_detrend=False, lat_name='lat',
                                  output_std=False, input_std=control_stdval, custom_std=True, first_threshold=100, last_threshold=450)

# grab numpy arrays
#psalt_nino = psalt_nino.values

Compute the variances.

In [12]:
# amoc collapse period (years 201-500)
print('amoc collapse period')
print(control_nino.sel(
    time=slice('0301-01-01 00:00:00','0600-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(g02sv_nino.sel(
    time=slice('0301-01-01 00:00:00','0600-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(g04sv_nino.sel(
    time=slice('0301-01-01 00:00:00','0600-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(p02sv_nino.sel(
    time=slice('0301-01-01 00:00:00','0600-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(p04sv_nino.sel(
    time=slice('0301-01-01 00:00:00','0600-12-31 00:00:00')).var(skipna=True).drop('z_t').values)

amoc collapse period
1.0290683683148447
1.2321633055166374
1.1028303528575303
1.2550042519717188
1.2014058716534053


In [13]:
# amoc active period (years 551-650)
print('amoc overshoot period')
print(control_nino.sel(
    time=slice('0651-01-01 00:00:00','0750-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(g02sv_nino.sel(
    time=slice('0651-01-01 00:00:00','0750-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(g04sv_nino.sel(
    time=slice('0651-01-01 00:00:00','0750-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(p02sv_nino.sel(
    time=slice('0651-01-01 00:00:00','0750-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(p04sv_nino.sel(
    time=slice('0651-01-01 00:00:00','0750-12-31 00:00:00')).var(skipna=True).drop('z_t').values)

amoc overshoot period
1.189405447147128
0.8324619015607647
1.1325987753446527
0.7754308415938108
1.051623261698329


In [14]:
# amoc active period (years 651-800)
print('amoc overshoot period')
print(control_nino.sel(
    time=slice('0751-01-01 00:00:00','0900-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(g02sv_nino.sel(
    time=slice('0751-01-01 00:00:00','0900-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(g04sv_nino.sel(
    time=slice('0751-01-01 00:00:00','0900-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(p02sv_nino.sel(
    time=slice('0751-01-01 00:00:00','0900-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(p04sv_nino.sel(
    time=slice('0751-01-01 00:00:00','0900-12-31 00:00:00')).var(skipna=True).drop('z_t').values)

amoc overshoot period
0.9714009781335337
0.5461056435982696
0.6010298518422034
0.576257094125526
0.6020547061270839


In [15]:
# pmoc active period (years 101-250)
print('(psalt)')
print(control_nino.sel(
    time=slice('0201-01-01 00:00:00','0350-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(psalt_nino.sel(
    time=slice('0201-01-01 00:00:00','0350-12-31 00:00:00')).var(skipna=True).values)

(psalt)
0.9554016701350656
0.8876958377807859


In [16]:
# pmoc inactive period (years 251-300)
print('(psalt)')
print(control_nino.sel(
    time=slice('0351-01-01 00:00:00','0400-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(psalt_nino.sel(
    time=slice('0351-01-01 00:00:00','0400-12-31 00:00:00')).var(skipna=True).values)

(psalt)
1.0052461433918805
1.358997323444676


In [17]:
# pmoc inactive period (years 301-350)
print('(psalt)')
print(control_nino.sel(
    time=slice('0401-01-01 00:00:00','0450-12-31 00:00:00')).var(skipna=True).drop('z_t').values)
print(psalt_nino.sel(
    time=slice('0401-01-01 00:00:00','0450-12-31 00:00:00')).var(skipna=True).values)

(psalt)
0.9927815576874726
0.5413684188164253
